In [1]:
##DEPENDENCIES
import pandas as pd
import datetime
from datetime import timedelta, date
import yfinance as yf
import numpy as np
import yahoofinancials
from yahoofinancials import YahooFinancials

In [2]:
##LIST OF DATES
#define function daterange
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
dates = []
#defne start and end date
#note: end date is day after last day of data
start_date = date(2020, 4, 1)
end_date = date(2020, 7, 1)
#loop through all dates, adding each to list
for single_date in daterange(start_date, end_date):
    curr_date = single_date.strftime("%Y-%m-%d")
    dates.append(curr_date)  

#define quarter ex. 2019Q1
q = '2020Q2'

In [3]:
##SCRAPE YAHOO FOR EARNINGS DATA

#creating empty data frame for raw data
column_names = ['Symbol', 'Company', 'Earnings Call Time', 'EPS Estimate', 'Reported EPS', 'Surprise(%)', 'Earnings Date']
combined_df = pd.DataFrame(columns = column_names)

#define start and end date
start = start_date.strftime("%Y-%m-%d")
end = end_date.strftime("%Y-%m-%d")
#loop through each day
i=0
while i < len(dates):
    day = dates[i]
    url = 'https://finance.yahoo.com/calendar/earnings?from='+start+'&to='+end+'&day='+day
    try:
        tables = pd.read_html(url)
    except:
        i=i+1
    else:
        df = tables[0]
        df.columns = ['Symbol', 'Company', 'Earnings Call Time', 'EPS Estimate', 'Reported EPS', 'Surprise(%)']
        j=0
        earn_dates = []
        while j < len(df['Symbol']):
            earn_dates.append(day)
            j=j+1
        df['Earnings Date'] = earn_dates
        combined_df = combined_df.append(df)
        #check if more than 100 symbols 
        if len(df['Symbol']) > 99:
            try:
                tables1 = pd.read_html(url+'&offset=100&size=100')
            except:
                i=i+1
            else:
                df1 = tables1[0]
                df1.columns = ['Symbol', 'Company', 'Earnings Call Time', 'EPS Estimate', 'Reported EPS', 'Surprise(%)']
                j=0
                earn_dates1 = []
                while j < len(df1['Symbol']):
                    earn_dates1.append(day)
                    j=j+1
                df1['Earnings Date'] = earn_dates1
                combined_df = combined_df.append(df1)
                #check if more than 200 symbols
                if len(df1['Symbol']) > 99:
                    try:
                        tables2 = pd.read_html(url+'&offset=200&size=100')
                    except:
                        i=i+1
                    else:
                        df2 = tables2[0]
                        df2.columns = ['Symbol', 'Company', 'Earnings Call Time', 'EPS Estimate', 'Reported EPS', 'Surprise(%)']
                        j=0
                        earn_dates2 = []
                        while j < len(df2['Symbol']):
                            earn_dates2.append(day)
                            j=j+1
                        df2['Earnings Date'] = earn_dates2
                        combined_df = combined_df.append(df2)
                        i=i+1
                else:
                    i=i+1
        else:
            i=i+1   
#raw data sent to .csv file
#note: rename output file for current quarter
combined_df = combined_df.sort_values(by=['Symbol', 'Earnings Date'])
combined_df.to_csv('resources/'+q+'_raw_earnings.csv')
combined_df.head()

,Symbol,Company,Earnings Call Time,EPS Estimate,Reported EPS,Surprise(%),Earnings Date
3,A,Agilent Technologies Inc,After Market Close,0.62,NaN,NaN,2020-05-21
0,AA,Alcoa Corp,TAS,-0.24,-0.23,4.17,2020-04-22
9,AABA,Altaba Inc,After Market Close,NaN,NaN,NaN,2020-05-01
8,AAL,American Airlines Group Inc,Before Market Open,-2.26,NaN,NaN,2020-04-30
8,AAMC,Altisource Asset Management Corp,Before Market Open,NaN,NaN,NaN,2020-05-06


In [4]:
##Scrape wikipedia for S&P 500 symbols
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
sp_symbols = list(df['Symbol'])
sp_symbols = sorted(sp_symbols)

In [5]:
##CREATE DICTIONARIES FOR EACH S&P 500 STOCK
#define lists
earnings_dates = []
multiples = []
sym_row = []
raw_symbols = list(combined_df['Symbol'])
unique_raw = []
for x in raw_symbols:
    if x not in unique_raw: 
        unique_raw.append(x)
#loop through S&P 500 symbols        
for sym in sp_symbols:
    #if no data for symbol
    if sym not in raw_symbols:
        earn_dict = {'Symbol' : sym,
                    q+' Time' : 'NaN',
                    q+' Date' : 'NaN'}  
    #if data for symbol    
    else:
        sym_row = [sym]
        for row in combined_df.iterrows():
            if row[1]['Symbol'] == sym:
                sym_row[0] = sym
                sym_row.append(row[1]['Earnings Call Time'])
                sym_row.append(row[1]['Earnings Date'])
        #if symbol has multiple dates        
        if len(sym_row) > 3:
            earn_dict = {'Symbol' : sym,
                    q+' Time' : 'Multi',
                    q+' Date' : 'Multi'}
            #create list of symbols with multiple dates (with the dates included)
            multiples.append(sym_row)
        #if symbol has only one date    
        else:
            earn_dict = {'Symbol' : sym_row[0],
                        q+' Time' : sym_row[1],
                        q+' Date' : sym_row[2]}        
    earnings_dates.append(earn_dict)
earnings_dates

[{'Symbol': 'A',
  '2020Q2 Time': 'After Market Close',
  '2020Q2 Date': '2020-05-21'},
 {'Symbol': 'AAL',
  '2020Q2 Time': 'Before Market Open',
  '2020Q2 Date': '2020-04-30'},
 {'Symbol': 'AAP',
  '2020Q2 Time': 'Before Market Open',
  '2020Q2 Date': '2020-05-20'},
 {'Symbol': 'AAPL',
  '2020Q2 Time': 'After Market Close',
  '2020Q2 Date': '2020-04-30'},
 {'Symbol': 'ABBV',
  '2020Q2 Time': 'Before Market Open',
  '2020Q2 Date': '2020-05-01'},
 {'Symbol': 'ABC',
  '2020Q2 Time': 'Before Market Open',
  '2020Q2 Date': '2020-05-07'},
 {'Symbol': 'ABMD',
  '2020Q2 Time': 'Before Market Open',
  '2020Q2 Date': '2020-04-30'},
 {'Symbol': 'ABT', '2020Q2 Time': 'TAS', '2020Q2 Date': '2020-04-16'},
 {'Symbol': 'ACN',
  '2020Q2 Time': 'Before Market Open',
  '2020Q2 Date': '2020-06-25'},
 {'Symbol': 'ADBE',
  '2020Q2 Time': 'After Market Close',
  '2020Q2 Date': '2020-06-11'},
 {'Symbol': 'ADI', '2020Q2 Time': 'TAS', '2020Q2 Date': '2020-05-20'},
 {'Symbol': 'ADM',
  '2020Q2 Time': 'After Mar

In [6]:
#S&P 500 stocks with multiple earnings dates in same quarter
mdf = pd.DataFrame(multiples)
mdf.to_csv('resources/'+q+'_sp_multi_earnings.csv')
mdf

,0,1,2,3,4,5,6,7,8
0,AON,TAS,2020-05-01,TAS,2020-05-01,None,None,None,None
1,BAC,Before Market Open,2020-04-15,TAS,2020-04-23,None,None,None,None
2,CTVA,Before Market Open,2020-05-07,Before Market Open,2020-05-07,None,None,None,None
3,DHR,TAS,2020-05-07,TAS,2020-05-07,None,None,None,None
4,EMR,TAS,2020-04-21,TAS,2020-04-21,None,None,None,None
5,EXPE,Time Not Supplied,2020-04-30,Time Not Supplied,2020-04-30,None,None,None,None
6,HWM,TAS,2020-04-14,TAS,2020-05-05,None,None,None,None
7,KMX,TAS,2020-04-02,Before Market Open,2020-06-19,None,None,None,None
8,MOS,Time Not Supplied,2020-05-04,After Market Close,2020-05-04,TAS,2020-05-05,None,None
9,NBL,Before Market Open,2020-05-01,Before Market Open,2020-05-08,None,None,None,None


In [7]:
#Data frame for all S&P 500 stocks
sp_quarter = pd.DataFrame(earnings_dates)
sp_quarter = sp_quarter[['Symbol', q+' Time', q+' Date']]
sp_quarter.to_csv('resources/'+q+'_sp_earnings.csv')
sp_quarter.head()

,Symbol,2020Q2 Time,2020Q2 Date
0,A,After Market Close,2020-05-21
1,AAL,Before Market Open,2020-04-30
2,AAP,Before Market Open,2020-05-20
3,AAPL,After Market Close,2020-04-30
4,ABBV,Before Market Open,2020-05-01
